In [1]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import datetime
from enum import Enum
from pydantic import BaseModel
from typing import List
from collections import defaultdict

# -----------------------------------------------------------------------------
# 1. 네임스페이스 등록
# -----------------------------------------------------------------------------
# HWPML 문서에서 사용하는 네임스페이스를 등록합니다.
NAMESPACES = {
    'ha': 'http://www.hancom.co.kr/hwpml/2011/app',
    'hp': 'http://www.hancom.co.kr/hwpml/2011/paragraph',
    'hp10': 'http://www.hancom.co.kr/hwpml/2016/paragraph',
    'hs': 'http://www.hancom.co.kr/hwpml/2011/section',
    'hc': 'http://www.hancom.co.kr/hwpml/2011/core',
    'hh': 'http://www.hancom.co.kr/hwpml/2011/head',
    'hhs': 'http://www.hancom.co.kr/hwpml/2011/history',
    'hm': 'http://www.hancom.co.kr/hwpml/2011/master-page',
    'hpf': 'http://www.hancom.co.kr/schema/2011/hpf',
    'dc': 'http://purl.org/dc/elements/1.1/',
    'opf': 'http://www.idpf.org/2007/opf/',
    'ooxmlchart': 'http://www.hancom.co.kr/hwpml/2016/ooxmlchart',
    'hwpunitchar': 'http://www.hancom.co.kr/hwpml/2016/HwpUnitChar',
    'epub': 'http://www.idpf.org/2007/ops',
    'config': 'urn:oasis:names:tc:opendocument:xmlns:config:1.0'
}

for prefix, uri in NAMESPACES.items():
    ET.register_namespace(prefix, uri)

# -----------------------------------------------------------------------------
# 2. 데이터 모델 정의
# -----------------------------------------------------------------------------

class FileType(str, Enum):
    """파일 타입
    파일이 실행 파일인지, 프로젝트 파일인지, 소스코드인지 타입으로 분류하는 enum 클래스.
    """
    EXECUTION = "실행파일"
    CONF = "환경파일"
    DB = "DB파일"
    PROJECT = "프로젝트 파일"
    SOURCE = "소스코드"
    IMAGE = "이미지 파일"
    UNKNOWN = "기타 파일"

class FileData(BaseModel):
    """파일 데이터
    파일 정보를 구성하는 클래스
    """
    Device: str
    Csu: str
    Type: FileType
    Index: int
    FilePath: str
    Filename: str
    Version: str
    Size: int
    Checksum: str
    Date: str
    PartNumber: str
    Loc: str
    Description: str

# -----------------------------------------------------------------------------
# 3. 샘플 데이터 정의
# -----------------------------------------------------------------------------
data = [
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.CONF,"Index":1,"FilePath":"temp\\MC_ADAgent","Filename":".gitignore","Version":"1.0","Size":121,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":2,"FilePath":"temp\\MC_ADAgent","Filename":"anomalyagent","Version":"1.0","Size":8044,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"33","Description":"이상 탐지 Anomalyagent 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":3,"FilePath":"temp\\MC_ADAgent","Filename":"inference","Version":"1.0","Size":7444,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"17","Description":"Anomalyagent 패킷 데이터 추론 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":4,"FilePath":"temp\\MC_ADAgent","Filename":"internal_ad","Version":"1.0","Size":5457,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"15","Description":"Anomalyagent 위협 신호 추론 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.EXECUTION,"Index":5,"FilePath":"temp\\MC_ADAgent","Filename":"mc_adagent","Version":"1.0","Size":329,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.CONF,"Index":6,"FilePath":"temp\\MC_ADAgent","Filename":"README","Version":"1.0","Size":625,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.EXECUTION,"Index":7,"FilePath":"temp\\MC_ADAgent","Filename":"requirements","Version":"1.0","Size":212,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":1,"FilePath":"temp\\MC_ADAgent\\app","Filename":"file_tailer","Version":"1.0","Size":12123,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"102","Description":"파일 tailing 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":2,"FilePath":"temp\\MC_ADAgent\\app","Filename":"__init__","Version":"1.0","Size":542,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"0","Description":"Logger 초기화"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":1,"FilePath":"temp\\MC_ADAgent\\app\\internal","Filename":"exception","Version":"1.0","Size":185,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"2","Description":"예외 클래스 모듈"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.SOURCE,"Index":2,"FilePath":"temp\\MC_ADAgent\\app\\internal","Filename":"logger","Version":"1.0","Size":3096,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"10","Description":"로거 등록 및 설정"},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.PROJECT,"Index":1,"FilePath":"temp\\MC_ADAgent\\app\\model","Filename":"model","Version":"1.0","Size":5282,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"47","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.PROJECT,"Index":2,"FilePath":"temp\\MC_ADAgent\\app\\model","Filename":"pca","Version":"1.0","Size":1699,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"62","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.DB,"Index":3,"FilePath":"temp\\MC_ADAgent\\app\\model","Filename":"scaler","Version":"1.0","Size":892,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.DB,"Index":1,"FilePath":"temp\\MC_ADAgent\\app\\model\\gru","Filename":"model_gru_364","Version":"1.0","Size":98072,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
    {"Device":"HDEV-001","Csu":"ADAgent","Type":FileType.DB,"Index":2,"FilePath":"temp\\MC_ADAgent\\app\\model\\gru","Filename":"scaler_gru","Version":"1.0","Size":769,"Checksum":"11","Date":"2025-05-25","PartNumber":"Q45019224E","Loc":"","Description":""},
]

file_data_list: List[FileData] = [FileData(**item) for item in data]

In [2]:
exe_files = sorted(
    [file for file in file_data_list if file.Type in {FileType.EXECUTION, FileType.CONF, FileType.DB}],
    key=lambda x: x.FilePath
)

prj_files = sorted(
    [file for file in file_data_list if file.Type in {FileType.PROJECT}],
    key=lambda x: x.FilePath
)

src_files = sorted(
    [file for file in file_data_list if file.Type in {FileType.SOURCE, FileType.IMAGE}],
    key=lambda x: x.FilePath
)

unknown_files = sorted(
    [file for file in file_data_list if file.Type in {FileType.UNKNOWN}],
    key=lambda x: x.FilePath
)

In [3]:
def create_p(parent, text, para_pr_id="13", style_id="2", char_pr_id="3"):
    """주어진 텍스트로 문단(<hp:p>) 요소를 생성합니다."""
    p_elem = ET.SubElement(parent, 'hp:p', {'paraPrIDRef': para_pr_id, 'styleIDRef': style_id})
    run_elem = ET.SubElement(p_elem, 'hp:run', {'charPrIDRef': char_pr_id})
    t_elem = ET.SubElement(run_elem, 'hp:t')
    t_elem.text = text
    return p_elem

def create_table_cell(parent, text, attrs):
    """표의 셀(<hp:tc>)을 생성합니다."""
    tc_attrs = {k: str(v) for k, v in attrs.get('tc', {}).items()}
    p_attrs = {k: str(v) for k, v in attrs.get('p', {}).items()}
    run_attrs = {k: str(v) for k, v in attrs.get('run', {}).items()}
    cellsz_attrs = {k: str(v) for k, v in attrs.get('cellsz', {}).items()}

    tc_elem = ET.SubElement(parent, 'hp:tc', tc_attrs)
    sub_list = ET.SubElement(tc_elem, 'hp:subList', {'vertAlign': 'CENTER'})
    p_elem = ET.SubElement(sub_list, 'hp:p', p_attrs)
    run_elem = ET.SubElement(p_elem, 'hp:run', run_attrs)
    t_elem = ET.SubElement(run_elem, 'hp:t')
    t_elem.text = text if text else ""
    ET.SubElement(tc_elem, 'hp:cellSz', cellsz_attrs)
    if 'cellspan' in attrs:
        ET.SubElement(tc_elem, 'hp:cellSpan', {k: str(v) for k, v in attrs['cellspan'].items()})
    return tc_elem

def create_executable_file_list_table(parent, data_groups):
    """'실행파일 목록' 표를 생성합니다."""
    p_for_tbl = ET.SubElement(parent, 'hp:p', {'paraPrIDRef': '13', 'styleIDRef': '2'})
    run_for_tbl = ET.SubElement(p_for_tbl, 'hp:run', {'charPrIDRef': '3'})
    
    total_files = sum(len(group['files']) for group in data_groups)
    total_rows = total_files + len(data_groups) + 1
    tbl_attr = {'id': '1868005351', 'rowCnt': str(total_rows), 'colCnt': '9', 'borderFillIDRef': '5'}
    tbl_elem = ET.SubElement(run_for_tbl, 'hp:tbl', tbl_attr)
    ET.SubElement(tbl_elem, 'hp:sz', {'width': '43534', 'height': '57300'})
    
    caption_p = ET.SubElement(ET.SubElement(ET.SubElement(tbl_elem, 'hp:caption', {'side': 'TOP'}), 'hp:subList'), 'hp:p', {'paraPrIDRef': '1', 'styleIDRef': '3'})
    caption_run = ET.SubElement(caption_p, 'hp:run', {'charPrIDRef': '2'})
    ET.SubElement(caption_run, 'hp:t').text = '표 1 실행파일 목록'

    tr_header = ET.SubElement(tbl_elem, 'hp:tr')
    headers = [
        ('구 분', {'tc': {'header': '1', 'borderFillIDRef': '7'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '4481', 'height': '1100'}}),
        ('순번', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '3231', 'height': '1100'}}),
        ('파일명', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '4365', 'height': '1100'}}),
        ('버전', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '3254', 'height': '1100'}}),
        ('크기 (Byte)', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '4229', 'height': '282'}}),
        ('첵섬', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '6936', 'height': '282'}}),
        ('수정일', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '4456', 'height': '282'}}),
        ('SW부품번호', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '5436', 'height': '282'}}),
        ('기능 설명', {'tc': {'borderFillIDRef': '9'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '7146', 'height': '282'}}),
    ]
    for text, attrs in headers:
        create_table_cell(tr_header, text, attrs)

    cell_attrs = {'tc': {'borderFillIDRef': '6'}, 'p': {'paraPrIDRef': '4', 'styleIDRef': '6'}, 'run': {'charPrIDRef': '0'}, 'cellsz': {'height': '1100'}}
    for section in data_groups:
        tr_loc = ET.SubElement(tbl_elem, 'hp:tr')
        create_table_cell(tr_loc, f"저장위치: {section['location']}", {'tc': {'borderFillIDRef': '10'}, 'p': {'paraPrIDRef': '4', 'styleIDRef': '6'}, 'run': {'charPrIDRef': '0'}, 'cellsz': {'width': '43534', 'height': '1100'}, 'cellspan': {'colSpan': '9'}})
        
        for file_info in section['files']:
            tr_file = ET.SubElement(tbl_elem, 'hp:tr')
            create_table_cell(tr_file, file_info.Type.value, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '4481'}})
            create_table_cell(tr_file, str(file_info.Index), {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '3231'}})
            create_table_cell(tr_file, file_info.Filename, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '4365'}})
            create_table_cell(tr_file, file_info.Version, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '3254'}})
            create_table_cell(tr_file, str(file_info.Size), {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '4229'}})
            create_table_cell(tr_file, file_info.Checksum, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '6936'}})
            create_table_cell(tr_file, file_info.Date, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '4456'}})
            create_table_cell(tr_file, file_info.PartNumber, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '5436'}})
            create_table_cell(tr_file, file_info.Description, {**cell_attrs, 'tc': {'borderFillIDRef': '13'}, 'cellsz': {**cell_attrs['cellsz'], 'width': '7146'}})
            
    return tbl_elem

def create_source_file_list_table(parent, data_groups):
    """'원본(소스)파일 목록' 표를 생성합니다."""
    p_for_tbl = ET.SubElement(parent, 'hp:p', {'paraPrIDRef': '18', 'styleIDRef': '0'})
    run_for_tbl = ET.SubElement(p_for_tbl, 'hp:run', {'charPrIDRef': '11'})
    
    total_files = sum(len(d['files']) for d in data_groups)
    total_rows = total_files + len(data_groups) + 1
    tbl_attr = {'id': '1868005372', 'rowCnt': str(total_rows), 'colCnt': '8', 'borderFillIDRef': '5'}
    tbl_elem = ET.SubElement(run_for_tbl, 'hp:tbl', tbl_attr)
    ET.SubElement(tbl_elem, 'hp:sz', {'width': '43310', 'height': '29700'})
    
    caption_p = ET.SubElement(ET.SubElement(ET.SubElement(tbl_elem, 'hp:caption', {'side': 'TOP'}), 'hp:subList'), 'hp:p', {'paraPrIDRef': '1', 'styleIDRef': '3'})
    caption_run = ET.SubElement(caption_p, 'hp:run', {'charPrIDRef': '2'})
    ET.SubElement(caption_run, 'hp:t').text = '표 3 원본(소스)파일 목록'

    tr_header = ET.SubElement(tbl_elem, 'hp:tr')
    headers = [
        ('순번', {'tc': {'header': '1', 'borderFillIDRef': '7'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '2780', 'height': '1100'}}),
        ('파일명', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '4555', 'height': '282'}}),
        ('버전', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '2330', 'height': '282'}}),
        ('크기 (Byte)', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '3951', 'height': '282'}}),
        ('첵섬', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '7018', 'height': '282'}}),
        ('생성일자', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '3651', 'height': '282'}}),
        ('라인수', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '3653', 'height': '282'}}),
        ('기능 설명', {'tc': {'borderFillIDRef': '9'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '1'}, 'cellsz': {'width': '15372', 'height': '282'}})
    ]
    for text, attrs in headers:
        create_table_cell(tr_header, text, attrs)
    
    cell_attrs = {'tc': {'borderFillIDRef': '6'}, 'p': {'paraPrIDRef': '4', 'styleIDRef': '6'}, 'run': {'charPrIDRef': '0'}, 'cellsz': {'height': '1100'}}
    for section in data_groups:
        tr_loc = ET.SubElement(tbl_elem, 'hp:tr')
        create_table_cell(tr_loc, f"저장위치: {section['location']}", {'tc': {'borderFillIDRef': '10'}, 'p': {'paraPrIDRef': '4', 'styleIDRef': '6'}, 'run': {'charPrIDRef': '0'}, 'cellsz': {'width': '43310', 'height': '1100'}, 'cellspan': {'colSpan': '8'}})

        for file_info in section['files']:
            tr_file = ET.SubElement(tbl_elem, 'hp:tr')
            create_table_cell(tr_file, str(file_info.Index), {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '2780'}})
            create_table_cell(tr_file, file_info.Filename, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '4555'}})
            create_table_cell(tr_file, file_info.Version, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '2330'}})
            create_table_cell(tr_file, str(file_info.Size), {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '3951'}})
            create_table_cell(tr_file, file_info.Checksum, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '7018'}})
            create_table_cell(tr_file, file_info.Date, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '3651'}})
            create_table_cell(tr_file, file_info.Loc, {**cell_attrs, 'cellsz': {**cell_attrs['cellsz'], 'width': '3653'}})
            create_table_cell(tr_file, file_info.Description, {**cell_attrs, 'tc': {'borderFillIDRef': '13'}, 'cellsz': {**cell_attrs['cellsz'], 'width': '15372'}})

    return tbl_elem

def create_misc_file_list_table(parent, data_groups):
    """'기타 파일 목록' 표를 생성합니다."""
    p_for_tbl = ET.SubElement(parent, 'hp:p', {'paraPrIDRef': '6', 'styleIDRef': '0'})
    run_for_tbl = ET.SubElement(p_for_tbl, 'hp:run', {'charPrIDRef': '36'})
    
    total_files = sum(len(d['files']) for d in data_groups)
    total_rows = total_files + len(data_groups) + 1
    tbl_attr = {'id': '1221686458', 'rowCnt': str(total_rows), 'colCnt': '7', 'borderFillIDRef': '4'}
    tbl_elem = ET.SubElement(run_for_tbl, 'hp:tbl', tbl_attr)
    ET.SubElement(tbl_elem, 'hp:sz', {'width': '40576', 'height': '14574'})
    
    caption_p = ET.SubElement(ET.SubElement(ET.SubElement(tbl_elem, 'hp:caption', {'side': 'TOP'}), 'hp:subList'), 'hp:p', {'paraPrIDRef': '5', 'styleIDRef': '3'})
    caption_run = ET.SubElement(caption_p, 'hp:run', {'charPrIDRef': '18'})
    ET.SubElement(caption_run, 'hp:t').text = '표13 기타 파일 목록'

    tr_header = ET.SubElement(tbl_elem, 'hp:tr')
    headers = [
        ('순번', {'tc': {'borderFillIDRef': '7'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '41'}, 'cellsz': {'width': '4669', 'height': '3243'}}),
        ('파일명', {'tc': {'borderFillIDRef': '9'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '41'}, 'cellsz': {'width': '8326', 'height': '3243'}}),
        ('버전', {'tc': {'borderFillIDRef': '19'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '41'}, 'cellsz': {'width': '4669', 'height': '3243'}}),
        ('크기 (Byte)', {'tc': {'borderFillIDRef': '36'}, 'p': {'paraPrIDRef': '23', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '41'}, 'cellsz': {'width': '4669', 'height': '3243'}}),
        ('첵섬', {'tc': {'borderFillIDRef': '36'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '41'}, 'cellsz': {'width': '4952', 'height': '3243'}}),
        ('수정일', {'tc': {'borderFillIDRef': '39'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '41'}, 'cellsz': {'width': '4669', 'height': '3243'}}),
        ('비고', {'tc': {'borderFillIDRef': '8'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '4'}, 'run': {'charPrIDRef': '41'}, 'cellsz': {'width': '8622', 'height': '3243'}})
    ]
    for text, attrs in headers:
        create_table_cell(tr_header, text, attrs)
        
    for section in data_groups:
        tr_loc = ET.SubElement(tbl_elem, 'hp:tr')
        create_table_cell(tr_loc, f"저장위치 : {section['location']}", {'tc': {'borderFillIDRef': '49'}, 'p': {'paraPrIDRef': '7', 'styleIDRef': '6'}, 'run': {'charPrIDRef': '2'}, 'cellsz': {'width': '40576', 'height': '2415'}, 'cellspan': {'colSpan': '7'}})

        for file_info in section['files']:
            tr_file = ET.SubElement(tbl_elem, 'hp:tr')
            create_table_cell(tr_file, str(file_info.Index), {'tc': {'borderFillIDRef': '33'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '5'}, 'run': {'charPrIDRef': '2'}, 'cellsz': {'width': '4669', 'height': '2892'}})
            create_table_cell(tr_file, file_info.Filename, {'tc': {'borderFillIDRef': '45'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '5'}, 'run': {'charPrIDRef': '2'}, 'cellsz': {'width': '8326', 'height': '2892'}})
            create_table_cell(tr_file, file_info.Version, {'tc': {'borderFillIDRef': '46'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '5'}, 'run': {'charPrIDRef': '2'}, 'cellsz': {'width': '4669', 'height': '2892'}})
            create_table_cell(tr_file, str(file_info.Size), {'tc': {'borderFillIDRef': '48'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '5'}, 'run': {'charPrIDRef': '2'}, 'cellsz': {'width': '4669', 'height': '2892'}})
            create_table_cell(tr_file, file_info.Checksum, {'tc': {'borderFillIDRef': '48'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '5'}, 'run': {'charPrIDRef': '37'}, 'cellsz': {'width': '4952', 'height': '2892'}})
            create_table_cell(tr_file, file_info.Date, {'tc': {'borderFillIDRef': '22'}, 'p': {'paraPrIDRef': '3', 'styleIDRef': '5'}, 'run': {'charPrIDRef': '2'}, 'cellsz': {'width': '4669', 'height': '2892'}})
            create_table_cell(tr_file, file_info.Description, {'tc': {'borderFillIDRef': '47'}, 'p': {'paraPrIDRef': '20', 'styleIDRef': '6'}, 'run': {'charPrIDRef': '2'}, 'cellsz': {'width': '8622', 'height': '2892'}})
    
    return tbl_elem


In [6]:

# -----------------------------------------------------------------------------
# 5. 메인 실행 로직
# -----------------------------------------------------------------------------
def main():
    """메인 함수: XML 템플릿을 읽고, 내용을 추가하여 새 파일로 저장합니다."""
    try:
        tree = ET.parse('section0.xml')
        root = tree.getroot()
    except FileNotFoundError:
        print("오류: 'section0.xml' 파일을 찾을 수 없습니다. 스크립트와 동일한 디렉토리에 있는지 확인하세요.")
        return
    except ET.ParseError as e:
        print(f"오류: 'section0.xml' 파일 파싱 중 오류 발생: {e}")
        return
        
    for child in list(root):
        root.remove(child)

    secpr_run = ET.SubElement(ET.SubElement(root, 'hp:p'), 'hp:run')
    secpr = ET.SubElement(secpr_run, 'hp:secPr')
    ET.SubElement(secpr, 'hp:pagePr', {'width': '59528', 'height': '84188'})
    
    # 데이터를 타입별, 경로별로 그룹화
    exec_groups = defaultdict(list)
    source_groups = defaultdict(list)
    misc_groups = defaultdict(list)

    exec_types = [FileType.EXECUTION, FileType.CONF, FileType.DB]
    source_types = [FileType.SOURCE, FileType.PROJECT]
    
    for item in file_data_list:
        if item.Type in exec_types:
            exec_groups[item.FilePath].append(item)
        elif item.Type in source_types:
            source_groups[item.FilePath].append(item)
        else: # UNKNOWN, IMAGE etc.
            misc_groups[item.FilePath].append(item)

    # 테이블 생성을 위해 그룹화된 데이터를 리스트 형태로 변환
    exec_data_for_table = [{'location': path, 'files': files} for path, files in exec_groups.items()]
    source_data_for_table = [{'location': path, 'files': files} for path, files in source_groups.items()]
    misc_data_for_table = [{'location': path, 'files': files} for path, files in misc_groups.items()]

    # --- 실행 파일 섹션 ---
    create_p(root, '실행파일', para_pr_id="15", style_id="1", char_pr_id="2")
    create_p(root, file_data_list[0].Device if file_data_list else "", para_pr_id="17", style_id="1", char_pr_id="2")
    total_exec_files = sum(len(files) for files in exec_groups.values())
    create_p(root, f'  ○ {file_data_list[0].Device}의 실행파일 총 수 : {total_exec_files}', para_pr_id="13", style_id="2", char_pr_id="3")
    create_p(root, '', para_pr_id="13", style_id="2", char_pr_id="3")
    if exec_data_for_table:
        create_executable_file_list_table(root, exec_data_for_table)
    
    # --- 원시 파일 섹션 ---
    create_p(root, '원시 파일', para_pr_id="21", style_id="1", char_pr_id="2")
    create_p(root, 'CSCI 형상항목 구성', para_pr_id="14", style_id="3", char_pr_id="2")
    create_p(root, file_data_list[0].Device if file_data_list else "", para_pr_id="16", style_id="1", char_pr_id="2")
    total_source_files = sum(len(files) for files in source_groups.values())
    create_p(root, f'  ○ {file_data_list[0].Device}의 원시파일 총 수 : {total_source_files}', para_pr_id="13", style_id="2", char_pr_id="3")
    create_p(root, '', para_pr_id="13", style_id="2", char_pr_id="3")
    create_p(root, file_data_list[0].Csu if file_data_list else "", para_pr_id="19", style_id="1", char_pr_id="2")
    if source_data_for_table:
        create_source_file_list_table(root, source_data_for_table)
    
    # --- 기타 파일 섹션 ---
    if misc_data_for_table:
        create_p(root, '기타 파일', para_pr_id="21", style_id="1", char_pr_id="2")
        create_misc_file_list_table(root, misc_data_for_table)

    # 생성된 XML을 보기 좋게 포맷팅
    rough_string = ET.tostring(root, 'utf-8', xml_declaration=True)
    reparsed = minidom.parseString(rough_string)
    pretty_xml_as_string = reparsed.toprettyxml(indent="  ", encoding="UTF-8").decode('utf-8')

    # 새 파일로 저장
    output_filename = 'section_generated_new.xml'
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(pretty_xml_as_string)

    print(f"'{output_filename}' 파일이 새로운 양식으로 성공적으로 생성되었습니다.")

if __name__ == '__main__':
    # pydantic이 설치되어 있어야 합니다. pip install pydantic
    main()

ExpatError: unbound prefix: line 3, column 4